In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from memory_fix import reduce_mem_usage
%pylab inline

import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [21]:
transactions_train = reduce_mem_usage( pd.read_csv('data/transactions_train.csv') )
X_test = reduce_mem_usage(pd.read_csv('data/transactions_test.csv'))

y_train = pd.read_csv('data/train_target.csv')
transactions_train['trans_date_times'] = transactions_train['trans_date'].apply(lambda x: get_date(x))

Memory usage of dataframe is 668.09 MB
Memory usage after optimization is: 250.53 MB
Decreased by 62.5%
Memory usage of dataframe is 668.38 MB
Memory usage after optimization is: 250.64 MB
Decreased by 62.5%


In [210]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [214]:
def get_date(day):
    first_jn = 61    
    return date(2018, 1, 1) + relativedelta(days = day - first_jn)

In [658]:
def get_diff(array):
    new_array = list()
    for i in range(1, len(array)):
        new_array.append(array[i] - array[i-1])
    
    return np.mean(new_array), np.max(new_array), np.min(new_array), np.std(new_array)

In [659]:
def poly_trend(timeseries, degree=1):
    """
    Поиск прямой, которая максимально соответствует тренду TS
    Возвращает коэффициенты линейной функции вида f(x) = ax + b
    """
    X = np.arange(0, len(timeseries))
    Y = np.array(timeseries)
    z = np.polyfit(X, Y, degree)

    return z[0],z[1]

In [660]:
def get_holiday_values(smb):
    counter = 0
    for i in smb.index:
        if holidays.RU().get(i):
            counter += 1  
    return counter

In [661]:
days = list(range(388))
dct = {}
for d in days:
    counter = 0
    user_date = get_date(d)
#     print(user_date)
    next_week = [user_date + relativedelta(days = i) for i in range(1, 8)]
    for i in next_week:
        
        if holidays.RU().get(i):
            counter += 1  
    dct[d] = counter

In [662]:
last_trans = transactions_train.groupby(['client_dk'])["trans_date"].agg(['max'])

In [663]:
def get_target_stat(transaction, target, is_train=True):
    stat = {'mean_date_diff':[], 'max_date_diff':[],'min_date_diff':[],'std_date_diff':[],
            'mean_sum_trans':[], 'mean_count_trans':[],'data_trend':[],
            'count_trans':[],'target':[], 'client_dk':[],
            'last_trans_year':[],'last_trans_month':[], 'last_trans_day':[],
            'first_trans_year':[],'first_trans_month':[], 'first_trans_day':[],'holiday_values':[],
            'user_last_trans_year':[], 'user_last_trans_month':[], 'user_last_trans_day':[],
            'holidays_next7':[]}
    
    small = transaction[transaction.small_group == target]
    group = small.groupby(['client_dk', 'trans_date_times'])["amount"].agg(['count', 'sum'])
    
    group_2 = small.groupby(['client_dk', 'trans_date'])["amount"].agg(['count', 'sum'])
    
    for ind in tqdm(transaction['client_dk'].unique()):
        last = last_trans.loc[ind]['max']
        
        stat['holidays_next7'].append(dct[last])
        last = get_date(int(last))


        if is_train:
            stat['target'].append(y_train[y_train.client_dk == ind][str(target)].values[0])
            stat['client_dk'].append(ind)
        else:
            stat['target'].append(-1)
            stat['client_dk'].append(ind)
            
        try:

            smb = group.loc[ind]
            smb_2 = group_2.loc[ind]
            mean, mx, mn, std = get_diff(list(smb_2.index))
#             print(mean, mx, mn, std)

            stat['mean_date_diff'].append(mean)
            stat['max_date_diff'].append(mx)
            stat['min_date_diff'].append(mn)
            stat['std_date_diff'].append(std)


            stat['mean_sum_trans'].append(np.mean(smb_2['sum']))
            stat['mean_count_trans'].append(np.mean(smb_2['count']))
            stat['count_trans'].append(len(smb))


            stat['last_trans_year'].append(smb.index[-1].year)
            stat['last_trans_month'].append(smb.index[-1].month)
            stat['last_trans_day'].append(smb.index[-1].day)
            
            
            stat['first_trans_year'].append(smb.index[0].year)
            stat['first_trans_month'].append(smb.index[0].month)
            stat['first_trans_day'].append(smb.index[0].day)
            
            stat['holiday_values'].append(get_holiday_values(smb) / len(smb))
            
            stat['user_last_trans_year'].append(last.year)
            stat['user_last_trans_month'].append(last.month)
            stat['user_last_trans_day'].append(last.day)
            
            
            
            
            try:
                date_line = poly_trend(list(smb.index))[0]
                stat['data_trend'].append(date_line)
            except Exception:
                stat['data_trend'].append(0)
            
            
        except Exception:
            stat['mean_date_diff'].append(-10000)
            stat['max_date_diff'].append(-10000)
            stat['min_date_diff'].append(-10000)
            stat['std_date_diff'].append(-10000)

            stat['mean_sum_trans'].append(-10000)
            stat['mean_count_trans'].append(-10000)
            stat['count_trans'].append(-10000)
                        
            stat['last_trans_year'].append(-10000)
            stat['last_trans_month'].append(-10000)
            stat['last_trans_day'].append(-10000)
            
            stat['first_trans_year'].append(-10000)
            stat['first_trans_month'].append(-10000)
            stat['first_trans_day'].append(-10000)
            
            stat['holiday_values'].append(-10000)


            


            stat['user_last_trans_year'].append(last.year)
            stat['user_last_trans_month'].append(last.month)
            stat['user_last_trans_day'].append(last.day)
            
            
            stat['data_trend'].append(-10000)

        
    return stat

In [664]:
TO_PREDICT = ['27', '32', '41', '45', '67', '73', '81', '88']

In [665]:
def create_files(transaction, is_train=True):
    for target in TO_PREDICT:
        stats = get_target_stat(transaction, target, is_train)
        df = pd.DataFrame(stats)
        pd.to_csv(df, f'cat_{target}_train={is_train}')

In [666]:
def predict(model):
    pred = []
    for ind, col in enumerate(TO_PREDICT):
        print(f"{ind+1} / {len(TO_PREDICT)}")
        
        data = get_target_stat(transactions_train, target=int(col))
        TRAIN = pd.DataFrame(data)
        model.fit(TRAIN.drop(['target', 'client_dk'], axis=1), TRAIN['target'])
        
        TEST = pd.DataFrame(get_target_stat(X_test, target=int(col), is_train=False))
        test_ids = TEST['client_dk']
        proba = model.predict_proba(TEST.drop(['target','client_dk'], axis=1))[:,1]
        pred.append(proba)
    
    return dict(zip(TO_PREDICT, pred)), test_ids

## Models

In [667]:
from sklearn.model_selection import cross_val_score
import catboost as cb
import eli5
import lightgbm

In [668]:
lgb = lightgbm.LGBMClassifier(learning_rate=0.025, n_estimators=1000,num_laeves=69,reg_lambda=3, random_seed=1007)


In [594]:
data_2 = get_target_stat(transactions_train, target=55, is_train=False)

In [603]:
data_2 = pd.DataFrame(data_2).add_prefix('c55_').rename(columns={'c55_client_dk':'client_dk'})

In [604]:
data = pd.DataFrame(data)

In [615]:
a = pd.concat([data, data_2], axis=1)

In [617]:
a.columns

Index(['mean_date_diff', 'max_date_diff', 'min_date_diff', 'std_date_diff',
       'mean_sum_trans', 'mean_count_trans', 'data_trend', 'count_trans',
       'target', 'client_dk', 'last_trans_year', 'last_trans_month',
       'last_trans_day', 'first_trans_year', 'first_trans_month',
       'first_trans_day', 'holiday_values', 'user_last_trans_year',
       'user_last_trans_month', 'user_last_trans_day', 'holidays_next7',
       'c55_mean_date_diff', 'c55_max_date_diff', 'c55_min_date_diff',
       'c55_std_date_diff', 'c55_mean_sum_trans', 'c55_mean_count_trans',
       'c55_data_trend', 'c55_count_trans', 'c55_target', 'client_dk',
       'c55_last_trans_year', 'c55_last_trans_month', 'c55_last_trans_day',
       'c55_first_trans_year', 'c55_first_trans_month', 'c55_first_trans_day',
       'c55_holiday_values', 'c55_user_last_trans_year',
       'c55_user_last_trans_month', 'c55_user_last_trans_day',
       'c55_holidays_next7'],
      dtype='object')

In [622]:
a = a.drop(['client_dk','c55_target'], axis=1)

In [580]:
def val(model):
    total_score = 0
    pred = []
    for ind, col in enumerate(TO_PREDICT):
        print(f"{ind+1}/{len(TO_PREDICT)}")
        
        data = get_target_stat(transactions_train, target=int(col))
        TRAIN = pd.DataFrame(data)
        score = cross_val_score(
            model,
            TRAIN.drop(['target', 'client_dk'], axis=1),
            TRAIN['target'],
            cv = 4,
            scoring = "roc_auc",
            verbose = 1
                      )
        total_score += np.mean(score)
        print(col, score, np.mean(score))
    print(total_score / 8)
val(cb.CatBoostClassifier())

1/8



Learning rate set to 0.038405


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.6578114	total: 18.2ms	remaining: 18.2s
1:	learn: 0.6256411	total: 40.6ms	remaining: 20.3s
2:	learn: 0.5974864	total: 57.4ms	remaining: 19.1s
3:	learn: 0.5710149	total: 76.7ms	remaining: 19.1s
4:	learn: 0.5473262	total: 93.8ms	remaining: 18.7s
5:	learn: 0.5259445	total: 111ms	remaining: 18.5s
6:	learn: 0.5071351	total: 138ms	remaining: 19.5s
7:	learn: 0.4899784	total: 154ms	remaining: 19.1s
8:	learn: 0.4741934	total: 175ms	remaining: 19.2s
9:	learn: 0.4600084	total: 195ms	remaining: 19.3s
10:	learn: 0.4476444	total: 213ms	remaining: 19.1s
11:	learn: 0.4359565	total: 234ms	remaining: 19.3s
12:	learn: 0.4253063	total: 259ms	remaining: 19.7s
13:	learn: 0.4162901	total: 278ms	remaining: 19.6s
14:	learn: 0.4077869	total: 296ms	remaining: 19.4s
15:	learn: 0.4003327	total: 315ms	remaining: 19.4s
16:	learn: 0.3934860	total: 334ms	remaining: 19.3s
17:	learn: 0.3874287	total: 349ms	remaining: 19s
18:	learn: 0.3815984	total: 367ms	remaining: 19s
19:	learn: 0.3771097	total: 385ms	remain

162:	learn: 0.3102023	total: 3.18s	remaining: 16.3s
163:	learn: 0.3102023	total: 3.19s	remaining: 16.3s
164:	learn: 0.3101653	total: 3.21s	remaining: 16.2s
165:	learn: 0.3101576	total: 3.22s	remaining: 16.2s
166:	learn: 0.3101486	total: 3.24s	remaining: 16.2s
167:	learn: 0.3101191	total: 3.26s	remaining: 16.2s
168:	learn: 0.3100923	total: 3.28s	remaining: 16.1s
169:	learn: 0.3099965	total: 3.3s	remaining: 16.1s
170:	learn: 0.3099572	total: 3.33s	remaining: 16.1s
171:	learn: 0.3099210	total: 3.35s	remaining: 16.1s
172:	learn: 0.3098738	total: 3.38s	remaining: 16.1s
173:	learn: 0.3098564	total: 3.4s	remaining: 16.1s
174:	learn: 0.3097483	total: 3.42s	remaining: 16.1s
175:	learn: 0.3097275	total: 3.44s	remaining: 16.1s
176:	learn: 0.3097003	total: 3.47s	remaining: 16.1s
177:	learn: 0.3096766	total: 3.49s	remaining: 16.1s
178:	learn: 0.3096020	total: 3.52s	remaining: 16.1s
179:	learn: 0.3095949	total: 3.54s	remaining: 16.1s
180:	learn: 0.3095247	total: 3.56s	remaining: 16.1s
181:	learn: 0.

...

434:	learn: 0.0714922	total: 9.06s	remaining: 11.8s
435:	learn: 0.0714874	total: 9.09s	remaining: 11.8s
436:	learn: 0.0714680	total: 9.11s	remaining: 11.7s
437:	learn: 0.0714653	total: 9.13s	remaining: 11.7s
438:	learn: 0.0714578	total: 9.15s	remaining: 11.7s
439:	learn: 0.0714449	total: 9.17s	remaining: 11.7s
440:	learn: 0.0714128	total: 9.19s	remaining: 11.7s
441:	learn: 0.0713924	total: 9.21s	remaining: 11.6s
442:	learn: 0.0713917	total: 9.23s	remaining: 11.6s
443:	learn: 0.0713868	total: 9.25s	remaining: 11.6s
444:	learn: 0.0713841	total: 9.28s	remaining: 11.6s
445:	learn: 0.0713670	total: 9.3s	remaining: 11.5s
446:	learn: 0.0713634	total: 9.32s	remaining: 11.5s
447:	learn: 0.0713257	total: 9.34s	remaining: 11.5s
448:	learn: 0.0712942	total: 9.36s	remaining: 11.5s
449:	learn: 0.0712921	total: 9.39s	remaining: 11.5s
450:	learn: 0.0712894	total: 9.41s	remaining: 11.5s
451:	learn: 0.0712663	total: 9.43s	remaining: 11.4s
452:	learn: 0.0712641	total: 9.45s	remaining: 11.4s
453:	learn: 0

597:	learn: 0.0693770	total: 12.4s	remaining: 8.35s
598:	learn: 0.0693753	total: 12.4s	remaining: 8.33s
599:	learn: 0.0693738	total: 12.5s	remaining: 8.31s
600:	learn: 0.0693675	total: 12.5s	remaining: 8.29s
601:	learn: 0.0693483	total: 12.5s	remaining: 8.27s
602:	learn: 0.0693210	total: 12.5s	remaining: 8.25s
603:	learn: 0.0693201	total: 12.5s	remaining: 8.22s
604:	learn: 0.0692973	total: 12.6s	remaining: 8.2s
605:	learn: 0.0692921	total: 12.6s	remaining: 8.18s
606:	learn: 0.0692917	total: 12.6s	remaining: 8.16s
607:	learn: 0.0692391	total: 12.6s	remaining: 8.14s
608:	learn: 0.0692383	total: 12.6s	remaining: 8.12s
609:	learn: 0.0692375	total: 12.7s	remaining: 8.1s
610:	learn: 0.0692235	total: 12.7s	remaining: 8.08s
611:	learn: 0.0692206	total: 12.7s	remaining: 8.05s
612:	learn: 0.0692204	total: 12.7s	remaining: 8.03s
613:	learn: 0.0692159	total: 12.7s	remaining: 8.01s
614:	learn: 0.0692159	total: 12.8s	remaining: 7.99s
615:	learn: 0.0692149	total: 12.8s	remaining: 7.96s
616:	learn: 0.

757:	learn: 0.0681903	total: 15.6s	remaining: 4.97s
758:	learn: 0.0681898	total: 15.6s	remaining: 4.95s
759:	learn: 0.0681894	total: 15.6s	remaining: 4.93s
760:	learn: 0.0681875	total: 15.6s	remaining: 4.91s
761:	learn: 0.0681792	total: 15.6s	remaining: 4.89s
762:	learn: 0.0681785	total: 15.7s	remaining: 4.87s
763:	learn: 0.0681785	total: 15.7s	remaining: 4.84s
764:	learn: 0.0681779	total: 15.7s	remaining: 4.82s
765:	learn: 0.0681775	total: 15.7s	remaining: 4.8s
766:	learn: 0.0681702	total: 15.7s	remaining: 4.78s
767:	learn: 0.0681683	total: 15.8s	remaining: 4.76s
768:	learn: 0.0681680	total: 15.8s	remaining: 4.74s
769:	learn: 0.0681627	total: 15.8s	remaining: 4.72s
770:	learn: 0.0681434	total: 15.8s	remaining: 4.7s
771:	learn: 0.0681433	total: 15.8s	remaining: 4.68s
772:	learn: 0.0681422	total: 15.9s	remaining: 4.66s
773:	learn: 0.0681412	total: 15.9s	remaining: 4.63s
774:	learn: 0.0681225	total: 15.9s	remaining: 4.62s
775:	learn: 0.0681090	total: 15.9s	remaining: 4.59s
776:	learn: 0.

919:	learn: 0.0673029	total: 18.7s	remaining: 1.62s
920:	learn: 0.0673012	total: 18.7s	remaining: 1.6s
921:	learn: 0.0672998	total: 18.7s	remaining: 1.58s
922:	learn: 0.0672759	total: 18.7s	remaining: 1.56s
923:	learn: 0.0672629	total: 18.8s	remaining: 1.54s
924:	learn: 0.0672618	total: 18.8s	remaining: 1.52s
925:	learn: 0.0672585	total: 18.8s	remaining: 1.5s
926:	learn: 0.0672528	total: 18.8s	remaining: 1.48s
927:	learn: 0.0672522	total: 18.8s	remaining: 1.46s
928:	learn: 0.0672511	total: 18.9s	remaining: 1.44s
929:	learn: 0.0672510	total: 18.9s	remaining: 1.42s
930:	learn: 0.0672425	total: 18.9s	remaining: 1.4s
931:	learn: 0.0672400	total: 18.9s	remaining: 1.38s
932:	learn: 0.0672387	total: 18.9s	remaining: 1.36s
933:	learn: 0.0672382	total: 19s	remaining: 1.34s
934:	learn: 0.0672345	total: 19s	remaining: 1.32s
935:	learn: 0.0672031	total: 19s	remaining: 1.3s
936:	learn: 0.0672028	total: 19s	remaining: 1.28s
937:	learn: 0.0672027	total: 19s	remaining: 1.26s
938:	learn: 0.0671988	tota

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.5min finished


In [ ]:
# def val(model):
#     pred = []
#     for ind, col in enumerate(TO_PREDICT):
#         print(f"{ind+1}/{len(TO_PREDICT)}")
        
#         data = get_target_stat(transactions_train, target=int(col))
#         TRAIN = pd.DataFrame(data)

#         score = cross_val_score(
#             model,
#             TRAIN.drop(['target', 'client_dk'], axis=1),
#             TRAIN['target'],
#             cv = 4,
#             scoring = "roc_auc",
#             verbose = 1
#                       )
#         print(col, score, np.mean(score))
# val(cb.CatBoostClassifier())

In [425]:
# prediction = predict(lgb)

## Submit

In [12]:
def submit(prediction, filename):
    pred, idx = prediction
    
    pred = pd.DataFrame(pred)
    pred['client_dk'] = idx
    submission = pd.DataFrame(data=np.zeros((25000,8)),columns=y_train.columns[1:],index=test_id['client_dk'].values)

    submission.index.name = 'client_dk'
    submission = submission.reset_index()
    
    sub = pd.DataFrame(submission['client_dk']).merge(pred, on='client_dk')
    
    sub = sub.set_index('client_dk')
    sub = sub.add_prefix("cat_")
    sub.to_csv(f'{filename}.csv')
    
    

In [272]:
def average_score(model, columns):
    for col in columns:
        score = cross_val_score(
        model,
        train_data,
        y_train[col],
        cv = 4,
        scoring = "roc_auc",
        verbose = 1,
        )
    print(col, score, np.mean(score))